1. Based on PostgreSQL tutorial https://www.postgresqltutorial.com/postgresql-python/ create PhoneBook:
 1) Design tables for PhoneBook.
 2) Implement two ways of inserting data into the PhoneBook.
    upload data from csv file.
    entering user name, phone from console.
 3) Implement updating data in the table (change user first name or phone).
 4) Querying data from the tables (with different filters).
 5) Implement deleting data from tables by username of phone.

In [12]:
import psycopg2
import csv

def get_connection():
    return psycopg2.connect(
        dbname="mydb",
        user="postgres",
        password="240407",
        host="localhost",
        port="5432"
    )

def create_table():
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS snake_game (
                    game_id SERIAL PRIMARY KEY,
                    player_name VARCHAR(50) NOT NULL,
                    score INT NOT NULL DEFAULT 0,
                    duration_seconds INT NOT NULL,
                    game_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            conn.commit()
            print("Table 'snake_game' has been created or already exists.")

def insert_from_csv(path):
    with get_connection() as conn:
        with conn.cursor() as cur:
            with open(path, mode='r', encoding='utf-8') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    cur.execute("""
                        INSERT INTO snake_game (game_id, player_name, score, duration_seconds, game_date)
                        VALUES (%s, %s, %s, %s, %s)
                    """, (int(row[0]), row[1].strip(), int(row[2]), int(row[3]), row[4]))
            conn.commit()
            print("Data successfully imported from CSV.")

def show_all():
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM snake_game ORDER BY score DESC;")
            rows = cur.fetchall()
            for row in rows:
                print(row)

def update_player():
    name = input("Enter the player name to update: ").strip()
    print("1. Change player name")
    print("2. Change score")
    choice = input("Your choice: ")

    with get_connection() as conn:
        with conn.cursor() as cur:
            if choice == '1':
                new_name = input("New player name: ").strip()
                cur.execute("UPDATE snake_game SET player_name = %s WHERE player_name = %s", (new_name, name))
            elif choice == '2':
                new_score = int(input("New score: "))
                cur.execute("UPDATE snake_game SET score = %s WHERE player_name = %s", (new_score, name))
            else:
                print("Invalid option.")
                return
            conn.commit()
            print("Update completed.")

def filter_players():
    print("1. Filter by name")
    print("2. Filter by score greater than X")
    print("3. Filter by score less than X")
    choice = input("Your choice: ")

    with get_connection() as conn:
        with conn.cursor() as cur:
            if choice == '1':
                name = input("Enter player name: ").strip()
                cur.execute("SELECT * FROM snake_game WHERE player_name = %s", (name,))
            elif choice == '2':
                value = int(input("Enter score X: "))
                cur.execute("SELECT * FROM snake_game WHERE score > %s", (value,))
            elif choice == '3':
                value = int(input("Enter score X: "))
                cur.execute("SELECT * FROM snake_game WHERE score < %s", (value,))
            else:
                print("Invalid option.")
                return

            results = cur.fetchall()
            if results:
                for row in results:
                    print(row)
            else:
                print("No matching results.")

def delete_player():
    name = input("Enter player name to delete: ").strip()
    with get_connection() as conn:
        with conn.cursor() as cur:
            cur.execute("DELETE FROM snake_game WHERE player_name = %s", (name,))
            conn.commit()
            print("Player deleted successfully.")

def menu():
    create_table()
    while True:
        print("\nSnake Game Menu:")
        print("1. Import from CSV")
        print("2. Show all records")
        print("3. Update player")
        print("4. Filter players")
        print("5. Delete player")
        print("0. Exit")
        choice = input("Your choice: ")

        if choice == '1':
            path = input("Enter path to CSV file: ")
            insert_from_csv(path)
        elif choice == '2':
            show_all()
        elif choice == '3':
            update_player()
        elif choice == '4':
            filter_players()
        elif choice == '5':
            delete_player()
        elif choice == '0':
            break
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    menu()


Table 'snake_game' has been created or already exists.

Snake Game Menu:
1. Import from CSV
2. Show all records
3. Update player
4. Filter players
5. Delete player
0. Exit
(21, ' Test1', 100, 50, datetime.datetime(2024, 4, 10, 0, 0))
(22, ' Test2', 100, 50, datetime.datetime(2024, 4, 10, 0, 0))

Snake Game Menu:
1. Import from CSV
2. Show all records
3. Update player
4. Filter players
5. Delete player
0. Exit
1. Change player name
2. Change score
Update completed.

Snake Game Menu:
1. Import from CSV
2. Show all records
3. Update player
4. Filter players
5. Delete player
0. Exit
